In [1]:
from models.yolo import Model

/home/hungnq/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hungnq/anaconda3/envs/torch/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## Model

In [ ]:
# import torch

# weights = "/home/hungnq/hungnq_2/yolov7/weights/yolov7.pt"
# ckpt = torch.load(weights, map_location="cpu")
# ckpt

In [1]:
# model = Model(
#     cfg='/home/hungnq/hungnq_2/yolov7/cfg/training/yolov7.yaml', 
#     ch=3, 
#     nc_per_head=[20,10], 
# )
# model 

In [11]:
a = True | False
a

True

In [ ]:
a =[0, 5, 2, 6, 8, 5, 1, 3]
a[-2:]

In [ ]:
from utils.torch_utils import intersect_dicts

state_dict = ckpt['model'].float().state_dict()  # to FP32
state_dict = intersect_dicts(state_dict, model.state_dict())  # intersect
model.load_state_dict(state_dict, strict=False)  # load

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
with open('data/hyp.scratch.mask.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)
weigths = torch.load('yolov7-mask.pt')
model = weigths['model']
model = model.half().to(device)
_ = model.eval()

## dataset

In [2]:
import torch
import os

from utils.torch_utils import ModelEMA, select_device, intersect_dicts, torch_distributed_zero_first, is_parallel
from utils.datasets import LoadImagesAndLabels, InfiniteDataLoader

def create_dataloader(path, imgsz, batch_size, stride, opt, hyp=None, augment=False, cache=False, pad=0.0, rect=False,
                      rank=-1, world_size=1, workers=8, image_weights=False, quad=False, prefix=''):
    # Make sure only the first process in DDP process the dataset first, and the following others can use the cache
    with torch_distributed_zero_first(rank):
        dataset = LoadImagesAndLabels(path, imgsz, batch_size,
                                      augment=augment,  # augment images
                                      hyp=hyp,  # augmentation hyperparameters
                                      rect=rect,  # rectangular training
                                      cache_images=cache,
                                      single_cls=opt['single_cls'],
                                      stride=int(stride),
                                      pad=pad,
                                      image_weights=image_weights,
                                      prefix=prefix)

    batch_size = min(batch_size, len(dataset))
    nw = min([os.cpu_count() // world_size, batch_size if batch_size > 1 else 0, workers])  # number of workers
    sampler = torch.utils.data.distributed.DistributedSampler(dataset) if rank != -1 else None
    loader = torch.utils.data.DataLoader if image_weights else InfiniteDataLoader
    # Use torch.utils.data.DataLoader() if dataset.properties will update during training else InfiniteDataLoader()
    dataloader = loader(dataset,
                        batch_size=batch_size,
                        num_workers=nw,
                        sampler=sampler,
                        pin_memory=True,
                        collate_fn=LoadImagesAndLabels.collate_fn4 if quad else LoadImagesAndLabels.collate_fn)
    return dataloader, dataset


In [3]:
opt = {
    'batch_size': 16,
    'data': 'data/coco.yaml',
    'hyp': '/home/hungnq/hungnq_2/yolov7/data/hyp.scratch.p5.yaml',
    'img_size': [640, 640],
    'multi_scale': False,
    'noautoanchor': False,
    'nosave': False,
    'notest': False,
    'rect': False,
    'resume': False,
    'single_cls': False,
    'world_size': 1
}

In [4]:
import yaml
from utils.general import colorstr


# Hyperparameters
with open(opt['hyp']) as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps

gs = 32

dataloader, dataset = create_dataloader(
    '/home/hungnq/hungnq_2/yolov7/coco/images', 
    640, 16, gs, 
    opt, hyp=hyp, augment=True, rect=False, rank=-1,
    world_size=1, workers=2, quad=False, prefix=colorstr('val: ')
)

val: Scanning '/home/hungnq/hungnq_2/yolov7/coco/labels/test2017.cache' images and labels... 122218 found, 41739 missing, 0 empty, 0 corrupted: 100%|██████████| 163957/163957 [00:00<?, ?it/s]


In [5]:
next(iter(dataloader))

[tensor([[[[180, 179, 180,  ...,  74,  73,  71],
           [181, 181, 182,  ...,  70,  68,  69],
           [183, 182, 183,  ...,  72,  72,  71],
           ...,
           [118,  91,  69,  ...,  68,  59,  63],
           [125, 111,  74,  ...,  66,  56,  57],
           [137, 134,  82,  ...,  62,  53,  50]],
 
          [[170, 169, 170,  ...,  77,  76,  74],
           [171, 170, 171,  ...,  74,  72,  72],
           [171, 170, 171,  ...,  76,  77,  75],
           ...,
           [116,  89,  66,  ...,  75,  65,  55],
           [123, 109,  71,  ...,  76,  66,  54],
           [134, 132,  79,  ...,  75,  66,  54]],
 
          [[160, 158, 158,  ...,  80,  78,  77],
           [161, 159, 159,  ...,  76,  74,  75],
           [161, 158, 159,  ...,  75,  76,  73],
           ...,
           [110,  82,  58,  ...,  70,  61,  55],
           [117, 101,  62,  ...,  72,  62,  51],
           [127, 124,  71,  ...,  72,  61,  47]]],
 
 
         [[[ 98,  97,  92,  ..., 114, 119, 127],
         